In [2]:
%run default-imports.ipynb

Using TensorFlow backend.


In [3]:
def run_validation_experiment(params, test, model, imputer=None, scaler=None):        
    
    ''' if model contains features not available in the new dataset, we include them as zeros '''    
    for feature in params.get('model_features'):
        if feature not in test.columns:
            test[feature] = pd.Series(np.zeros(test.shape[0]))    
        
    ''' reorder columns in order to match the original order '''
    test = test[params['model_features']]
    
    ''' evaluate models '''
    test, _ = Impute().execute(test.copy(), imputer=imputer)    
    test,_ = Scale().execute(test.copy(), params['target'], scaler=scaler)    
    results = Evaluate().execute(test.copy(), target=params['target'], models={params['algorithm']:model})
    
    ''' obtain performance metrics '''
    performance = {}    
    performance['discrimination'] = get_discrimination_metrics(**results[params['algorithm']])
    performance['calibration'] = get_calibration_metrics(results[params['algorithm']]['y_true'],results[params['algorithm']]['y_probs'])
    performance['clinical_usefulness'] = get_clinical_usefulness_metrics(performance['discrimination'])    
    
    params['datetime'] = datetime.now()
    
    ''' summarize results '''
    experiment = {'parameters' : params,
                  'results': results[params['algorithm']],
                  'performance' : performance}
    return experiment
    
    